In [14]:
import pandas as pd
import numpy as np
import plotly.express as px

In [15]:
kuka_output = pd.read_excel('KukaOutput-ICNAP.xlsx', engine='openpyxl')
kuka_output['EndDate'] = pd.to_datetime(kuka_output['EndDate'])
t0 = kuka_output['EndDate'].iloc[0]
kuka_output['EndDate min'] = (kuka_output['EndDate'] - t0).astype('timedelta64[m]')
kuka_output.head(20)

,KukaRobotId,Product,StartDate,EndDate,EndDate min
0,68051,Rotor A3,2022-01-31 20:13:35,2022-02-01 01:01:47,0.0
1,68051,Rotor A3,2022-02-01 01:02:12,2022-02-01 01:02:55,1.0
2,82402,Rotor A4,2022-01-31 23:27:53,2022-02-01 03:09:23,127.0
3,82403,Rotor A2,2022-01-31 23:26:11,2022-02-01 03:18:29,136.0
4,68050,Rotor A3,2022-02-01 01:03:26,2022-02-01 04:20:59,199.0
5,68050,Rotor A3,2022-02-01 04:21:22,2022-02-01 06:40:31,338.0
6,68050,Rotor C1,2022-02-01 06:45:10,2022-02-01 06:45:43,343.0
7,82403,Rotor A2,2022-02-01 03:18:34,2022-02-01 07:07:44,365.0
8,82402,Rotor B3,2022-02-01 03:09:33,2022-02-01 09:46:59,525.0
9,82402,Rotor B3,2022-02-01 09:47:05,2022-02-01 09:53:56,532.0


In [16]:
programs = pd.read_excel('OvenPrograms-ICNAP.xlsx', engine='openpyxl')
programs.head()

,program name,product type,target temp (C),program duration (min),cooling temp (C),time to cooling (min)
0,A01 Kuk,Rotor A0,319.55085,260,42.60678,60
1,A01 Kuk,Rotor A1,319.55085,260,42.60678,60
2,A23 Kuk,Rotor A2,319.55085,390,42.60678,60
3,A23 Kuk,Rotor A3,319.55085,390,42.60678,60
4,A456 Kuk,Rotor A4,319.55085,400,42.60678,60


In [17]:
class Oven:
    def __init__(self, index, finish_time=0, temp=0, rotor_list=[]):
        self.index = index
        self.finish_time = finish_time
        self.temp = temp
        self.rotor_list = rotor_list
    
    def get_energy_cost(self, target_temp):
        return target_temp-self.temp
    
    def get_time_cost(self, time_available, duration):
        if time_available > self.finish_time:
            return time_available
        return self.finish_time

    def add_rotor(self, time_available, duration, cold_temp, index):
        self.temp = cold_temp
        self.rotor_list.append(index)
        if time_available > self.finish_time:
            self.finish_time = time_available + duration
            
        else:
            self.finish_time += duration
        
class Node:
    def __init__(self, path, remaining_df, ovens, node_dict):
        self.path = path
        self.remaining_df = remaining_df
        self.ovens = ovens
        self.node_dict = node_dict
        
    def get_next(self):
        best_try = self.get_try(0)
        return best_try
    
    def get_try(self, index):
        next_try = dict()
        next_try['product'] = self.remaining_df['Product'].iloc[index]
        next_try['time_available'] = self.remaining_df['EndDate min'].iloc[index]
        next_try['duration'] = programs[programs['product type']==next_try['product']]['program duration (min)'].values[0] + 60
        next_try['target_temp'] = programs[programs['product type']==next_try['product']]['target temp (C)'].values[0]
        next_try['cool_temp'] = programs[programs['product type']==next_try['product']]['cooling temp  (C)'].values[0]
        next_try['product_index'] = self.remaining_df.iloc[index].name
        time_costs = [oven.get_time_cost(next_try['time_available'], next_try['duration']) for oven in self.ovens]
        next_try['time_cost'] = min(time_costs)
        next_try['oven_index'] = time_costs.index(next_try['time_cost'])
        next_try['energy_cost'] = self.ovens[next_try['oven_index']].get_energy_cost(next_try['target_temp'])
        return next_try

In [18]:
oven_list = [Oven(i, finish_time=0, temp=0, rotor_list=[]) for i in range(6)]
path = []
df_remaining = kuka_output.copy()
current_node = Node(path, df_remaining, oven_list, {})


while df_remaining.shape[0]>0:
    next_node_dict = current_node.get_next()
    oven_list[next_node_dict['oven_index']].add_rotor(next_node_dict['time_available'], next_node_dict['duration'],
                                                      next_node_dict['cool_temp'], next_node_dict['product_index'])
    df_remaining.drop(index=[next_node_dict['product_index']], inplace=True)
    path.append(current_node)
    current_node = Node(path, df_remaining, oven_list, next_node_dict)
    if df_remaining.shape[0] == 0:
        path.append(current_node)

In [19]:
node_dicts = [i.node_dict for i in path][1:]
node_dicts

[{'product': 'Rotor A3',
  'time_available': 0.0,
  'duration': 450,
  'target_temp': 319.55085,
  'cool_temp': 42.60678,
  'product_index': 0,
  'time_cost': 0,
  'oven_index': 0,
  'energy_cost': 319.55085},
 {'product': 'Rotor A3',
  'time_available': 1.0,
  'duration': 450,
  'target_temp': 319.55085,
  'cool_temp': 42.60678,
  'product_index': 1,
  'time_cost': 1.0,
  'oven_index': 1,
  'energy_cost': 319.55085},
 {'product': 'Rotor A4',
  'time_available': 127.0,
  'duration': 460,
  'target_temp': 319.55085,
  'cool_temp': 42.60678,
  'product_index': 2,
  'time_cost': 127.0,
  'oven_index': 2,
  'energy_cost': 319.55085},
 {'product': 'Rotor A2',
  'time_available': 136.0,
  'duration': 450,
  'target_temp': 319.55085,
  'cool_temp': 42.60678,
  'product_index': 3,
  'time_cost': 136.0,
  'oven_index': 3,
  'energy_cost': 319.55085},
 {'product': 'Rotor A3',
  'time_available': 199.0,
  'duration': 450,
  'target_temp': 319.55085,
  'cool_temp': 42.60678,
  'product_index': 4,


In [20]:
nodes_df = pd.DataFrame(node_dicts)
nodes_df['end job'] = nodes_df['time_cost'] + nodes_df['duration']
nodes_df.head(20)

,product,time_available,duration,target_temp,cool_temp,product_index,time_cost,oven_index,energy_cost,end job
0,Rotor A3,0.0,450,319.55085,42.60678,0,0.0,0,319.55085,450.0
1,Rotor A3,1.0,450,319.55085,42.60678,1,1.0,1,319.55085,451.0
2,Rotor A4,127.0,460,319.55085,42.60678,2,127.0,2,319.55085,587.0
3,Rotor A2,136.0,450,319.55085,42.60678,3,136.0,3,319.55085,586.0
4,Rotor A3,199.0,450,319.55085,42.60678,4,199.0,4,319.55085,649.0
5,Rotor A3,338.0,450,319.55085,42.60678,5,338.0,5,319.55085,788.0
6,Rotor C1,343.0,240,333.75311,42.60678,6,450.0,0,291.14633,690.0
7,Rotor A2,365.0,450,319.55085,42.60678,7,451.0,1,276.94407,901.0
8,Rotor B3,525.0,300,326.65198,71.01130,8,586.0,3,284.04520,886.0
9,Rotor B3,532.0,300,326.65198,71.01130,9,587.0,2,284.04520,887.0


In [21]:
kuka_output.head(10)

,KukaRobotId,Product,StartDate,EndDate,EndDate min
0,68051,Rotor A3,2022-01-31 20:13:35,2022-02-01 01:01:47,0.0
1,68051,Rotor A3,2022-02-01 01:02:12,2022-02-01 01:02:55,1.0
2,82402,Rotor A4,2022-01-31 23:27:53,2022-02-01 03:09:23,127.0
3,82403,Rotor A2,2022-01-31 23:26:11,2022-02-01 03:18:29,136.0
4,68050,Rotor A3,2022-02-01 01:03:26,2022-02-01 04:20:59,199.0
5,68050,Rotor A3,2022-02-01 04:21:22,2022-02-01 06:40:31,338.0
6,68050,Rotor C1,2022-02-01 06:45:10,2022-02-01 06:45:43,343.0
7,82403,Rotor A2,2022-02-01 03:18:34,2022-02-01 07:07:44,365.0
8,82402,Rotor B3,2022-02-01 03:09:33,2022-02-01 09:46:59,525.0
9,82402,Rotor B3,2022-02-01 09:47:05,2022-02-01 09:53:56,532.0


In [22]:
programs.head()

,program name,product type,target temp (C),program duration (min),cooling temp (C),time to cooling (min)
0,A01 Kuk,Rotor A0,319.55085,260,42.60678,60
1,A01 Kuk,Rotor A1,319.55085,260,42.60678,60
2,A23 Kuk,Rotor A2,319.55085,390,42.60678,60
3,A23 Kuk,Rotor A3,319.55085,390,42.60678,60
4,A456 Kuk,Rotor A4,319.55085,400,42.60678,60


In [23]:
nodes_df['start_job_date'] = t0 + pd.to_timedelta(nodes_df['time_cost'], unit='m')
nodes_df['finish_job_date'] = t0 + pd.to_timedelta(nodes_df['end job'], unit='m')

nodes_df.head(30)

,product,time_available,duration,target_temp,cool_temp,product_index,time_cost,oven_index,energy_cost,end job,start_job_date,finish_job_date
0,Rotor A3,0.0,450,319.55085,42.60678,0,0.0,0,319.55085,450.0,2022-02-01 01:01:47,2022-02-01 08:31:47
1,Rotor A3,1.0,450,319.55085,42.60678,1,1.0,1,319.55085,451.0,2022-02-01 01:02:47,2022-02-01 08:32:47
2,Rotor A4,127.0,460,319.55085,42.60678,2,127.0,2,319.55085,587.0,2022-02-01 03:08:47,2022-02-01 10:48:47
3,Rotor A2,136.0,450,319.55085,42.60678,3,136.0,3,319.55085,586.0,2022-02-01 03:17:47,2022-02-01 10:47:47
4,Rotor A3,199.0,450,319.55085,42.60678,4,199.0,4,319.55085,649.0,2022-02-01 04:20:47,2022-02-01 11:50:47
5,Rotor A3,338.0,450,319.55085,42.60678,5,338.0,5,319.55085,788.0,2022-02-01 06:39:47,2022-02-01 14:09:47
6,Rotor C1,343.0,240,333.75311,42.60678,6,450.0,0,291.14633,690.0,2022-02-01 08:31:47,2022-02-01 12:31:47
7,Rotor A2,365.0,450,319.55085,42.60678,7,451.0,1,276.94407,901.0,2022-02-01 08:32:47,2022-02-01 16:02:47
8,Rotor B3,525.0,300,326.65198,71.01130,8,586.0,3,284.04520,886.0,2022-02-01 10:47:47,2022-02-01 15:47:47
9,Rotor B3,532.0,300,326.65198,71.01130,9,587.0,2,284.04520,887.0,2022-02-01 10:48:47,2022-02-01 15:48:47


In [24]:
fig = px.timeline(nodes_df, x_start='start_job_date', x_end='finish_job_date', y="oven_index", color="product", text="product_index")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

In [12]:
joined_df = kuka_output.join(nodes_df.set_index('product_index'), lsuffix='_caller', rsuffix='_other')
joined_df.head(20)

,KukaRobotId,Product,StartDate,EndDate,EndDate min,product,time_available,duration,target_temp,cool_temp,time_cost,oven_index,energy_cost,end job,start_job_date,finish_job_date
0,68051,Rotor A3,2022-01-31 20:13:35,2022-02-01 01:01:47,0.0,Rotor A3,0.0,450,319.55085,42.60678,0.0,0,319.55085,450.0,2022-02-01 01:01:47,2022-02-01 08:31:47
1,68051,Rotor A3,2022-02-01 01:02:12,2022-02-01 01:02:55,1.0,Rotor A3,1.0,450,319.55085,42.60678,1.0,1,319.55085,451.0,2022-02-01 01:02:47,2022-02-01 08:32:47
2,82402,Rotor A4,2022-01-31 23:27:53,2022-02-01 03:09:23,127.0,Rotor A4,127.0,460,319.55085,42.60678,127.0,2,319.55085,587.0,2022-02-01 03:08:47,2022-02-01 10:48:47
3,82403,Rotor A2,2022-01-31 23:26:11,2022-02-01 03:18:29,136.0,Rotor A2,136.0,450,319.55085,42.60678,136.0,3,319.55085,586.0,2022-02-01 03:17:47,2022-02-01 10:47:47
4,68050,Rotor A3,2022-02-01 01:03:26,2022-02-01 04:20:59,199.0,Rotor A3,199.0,450,319.55085,42.60678,199.0,4,319.55085,649.0,2022-02-01 04:20:47,2022-02-01 11:50:47
5,68050,Rotor A3,2022-02-01 04:21:22,2022-02-01 06:40:31,338.0,Rotor A3,338.0,450,319.55085,42.60678,338.0,5,319.55085,788.0,2022-02-01 06:39:47,2022-02-01 14:09:47
6,68050,Rotor C1,2022-02-01 06:45:10,2022-02-01 06:45:43,343.0,Rotor C1,343.0,240,333.75311,42.60678,450.0,0,291.14633,690.0,2022-02-01 08:31:47,2022-02-01 12:31:47
7,82403,Rotor A2,2022-02-01 03:18:34,2022-02-01 07:07:44,365.0,Rotor A2,365.0,450,319.55085,42.60678,451.0,1,276.94407,901.0,2022-02-01 08:32:47,2022-02-01 16:02:47
8,82402,Rotor B3,2022-02-01 03:09:33,2022-02-01 09:46:59,525.0,Rotor B3,525.0,300,326.65198,71.01130,586.0,3,284.04520,886.0,2022-02-01 10:47:47,2022-02-01 15:47:47
9,82402,Rotor B3,2022-02-01 09:47:05,2022-02-01 09:53:56,532.0,Rotor B3,532.0,300,326.65198,71.01130,587.0,2,284.04520,887.0,2022-02-01 10:48:47,2022-02-01 15:48:47


In [13]:
joined_df.tail()

,KukaRobotId,Product,StartDate,EndDate,EndDate min,product,time_available,duration,target_temp,cool_temp,time_cost,oven_index,energy_cost,end job,start_job_date,finish_job_date
526,82403,Rotor A1,2022-02-28 20:14:35,2022-02-28 23:24:19,40222.0,Rotor A1,40222.0,320,319.55085,42.60678,40561.0,1,276.94407,40881.0,2022-03-01 05:02:47,2022-03-01 10:22:47
527,82402,Rotor B3,2022-02-28 23:24:00,2022-03-01 01:14:49,40333.0,Rotor B3,40333.0,300,326.65198,71.01130,40705.0,3,284.04520,41005.0,2022-03-01 07:26:47,2022-03-01 12:26:47
528,82403,Rotor A1,2022-02-28 23:29:09,2022-03-01 02:45:57,40424.0,Rotor A1,40424.0,320,319.55085,42.60678,40719.0,2,276.94407,41039.0,2022-03-01 07:40:47,2022-03-01 13:00:47
529,68050,Rotor C1,2022-02-28 23:46:24,2022-03-01 04:13:09,40511.0,Rotor C1,40511.0,240,333.75311,42.60678,40758.0,5,262.74181,40998.0,2022-03-01 08:19:47,2022-03-01 12:19:47
530,82401,Rotor A5,2022-02-28 12:27:06,2022-03-01 06:17:10,40635.0,Rotor A5,40635.0,460,319.55085,42.60678,40837.0,0,248.53955,41297.0,2022-03-01 09:38:47,2022-03-01 17:18:47


In [27]:
nodes_df['waiting_time'] = (nodes_df['time_cost']-nodes_df['time_available'])
nodes_df[nodes_df['waiting_time']==703]

,product,time_available,duration,target_temp,cool_temp,product_index,time_cost,oven_index,energy_cost,end job,start_job_date,finish_job_date,waiting_time
189,Rotor A2,13210.0,450,319.55085,42.60678,189,13913.0,5,248.53955,14363.0,2022-02-10 16:54:47,2022-02-11 00:24:47,703.0


In [35]:
fig = px.histogram(nodes_df, x='waiting_time', title="Idle histogram FIFO")
fig.update_layout(yaxis_range=[0,300],xaxis_range=[0,4500])
fig.show()

In [32]:
(nodes_df['time_cost']-nodes_df['time_available']).median()

107.0